# Seq2Seqで文章生成
文章生成向けにRNNを使用したSequence to Sequence（Seq2Seq）が有名です。 最近の研究ではこちらに対して、Attentionという機構を追加したりと話はすでに進んでいますが、まずは基礎的なSeq2Seqの流れを把握しておきましょう。

Seq2Seqがこれまでの手法と大きく異なる点としては、入力が**可変長**、出力が**可変長**であることです。 そのため、固定長の入力から固定長の出力を得るような機械学習において、扱いずらい問題設定であることは明白です。

また、実装上の問題にもなってくるのですが、基本的にはNumpyでは（サンプル数, 入力変数の数）という形式でデータが格納されている必要がありますが、文章の長さが異なることもあり、入力変数の数が各サンプル事で異なってしまい、Numpyの行列として格納することができません。 その問題を解決するために実装上では、`F.concat`をつかってうまくその問題を回避しているので、そういった点も注目してみてください。

今回は[keras/examples](https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py)にある`addition_rnn.py` をもとに解説を進めます。 すべての完璧なコードはこちらにあるため、この解説でざっくりと計算の流れをつかんでから、本格的なコードに進むと良いかと思います。

## 必要なモジュールの読み込み

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## データセットの作成

今回は非常に簡単な問題設定ですが、`12+3`という文字列を入力すると、`15`という文字列が返ってくるような設定としましょう。
果たして機械が足し算という規則性を再現することができるのかといった問題設定です。

このインプットを日本語、アウトプットを英語にした場合は機械翻訳、インプットを質問、アウトプットを回答とした場合はチャットボットと呼ばれるものになります。

まずは足し算のデータセットを作っていきましょう。
実際の文章を適用する際のデータセットの形の参考になるため、どのような形式であるかをしっかり見ておきましょう。

In [2]:
# シードの固定
np.random.seed(1)

# サンプル数5000
x1 = np.random.randint(low=0, high=99, size=(5000,))
x2 = np.random.randint(low=0, high=99, size=(5000,))

In [3]:
questions, expected = [], []
for (_x1, _x2) in zip(x1, x2):
    questions.append( '{}+{}'.format(_x1, _x2) )
    expected.append( '{}'.format(_x1 + _x2) )

入力を`question`とし、出力を`expected`としましょう。
では、作成した文字列を数値に変換していきます。

### IDに変換

入力変数としては、単語単位で扱うのではなく、各単語に割り振られた要素番号（インデックス番号：ID）を使用します。
単語（word）をIDに変換するため `word2id`、また最終的に元に戻すために `id2word` を下記のように作っておくと便利です。

In [4]:
# 今回使用する文字列12種類
chars = '0123456789+ '

word2id = dict((c, i) for i, c in enumerate(chars))
id2word = dict((i, c) for i, c in enumerate(chars))

In [5]:
word2id

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 '+': 10,
 ' ': 11}

In [6]:
id2word

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '+',
 11: ' '}

では、作成したIDを元にデータを変換（エンコード）しましょう。`xs`, `ts`として入力と教師データをリストへまとめます。
なぜ、これまで使用していた`x`と`t`でないかというと、固定長の場合は`x`と`t`で、それが可変長となった場合は`x`の`sequence`ということで`xs`や`ts`という風に名付けています。

入力`xs`に関しては2桁x2に+を加えて5桁になります。また教師データは2桁の足し算なので最大で3桁となります。
それぞれの桁数に対して、IDの数だけ次元を用意します。

In [7]:
xs0 = np.zeros((5, 12))
xs0

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [8]:
sentence = questions[0]
sentence

'37+78'

作成した`xs0`にIDを割り振ります。  
まず`37+78`の先頭の値`3`のインデックスを表示しましょう。

In [9]:
word2id[sentence[0]]

3

インデックスの対応する位置に`1`を割り当てます。

In [10]:
xs0[0, word2id[sentence[0]]] = 1 # numpyのインデックス
xs0

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

対応するインデックス位置に`1`が割り振られましたね。  
このような形でIDの割り振りを繰り返していきます。

### 演習
では、先程までの流れを振り返りながら、   
`37+78`をエンコードしていきましょう。

In [11]:
for i, _c in enumerate(sentence):
    xs0[i, word2id[_c]] = 1

In [12]:
xs0

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

エンコード関数を作成します。

In [13]:
def encode(sentence, num_rows):
    x = np.zeros((num_rows, 12))
    for i, c in enumerate(sentence):
        x[i, word2id[c]] = 1
    return x

先ほどの例を確認しましょう。

In [14]:
encode(sentence, 5)

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

では全体の5000サンプルのデータ作成を行いましょう。

In [15]:
# 入力の最大長
MAXLEN = 5
# 入力の長さ
DIGITS = 2

# 入力の作成
xs = np.zeros((len(questions), MAXLEN, len(chars)))

# サンプル数、最大長、入力の種類
xs.shape

(5000, 5, 12)

In [16]:
xs = np.zeros((len(questions), MAXLEN, len(chars)))
ts = np.zeros((len(expected), DIGITS + 1, len(chars)))
for i, sentence in enumerate(questions):
    xs[i] = encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    ts[i] = encode(sentence, DIGITS + 1)

In [17]:
xs.shape

(5000, 5, 12)

In [18]:
ts.shape

(5000, 3, 12)

In [19]:
xs[0]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [20]:
ts[0]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

## 訓練データと検証データに分割

In [21]:
from sklearn.model_selection import train_test_split

# 訓練データと検証データの分割
train_x, val_x, train_t, val_t = train_test_split(xs, ts, train_size=0.9,  test_size=0.1, random_state=0, shuffle=False)

In [22]:
print('Training Data:')
print(train_x.shape)
print(train_t.shape)

print('Validation Data:')
print(val_x.shape)
print(val_t.shape)

Training Data:
(4500, 5, 12)
(4500, 3, 12)
Validation Data:
(500, 5, 12)
(500, 3, 12)


## モデルの構築
### モデルの定義（stateless）
RNNはステートフル（stateful）であり、学習中にバッチ間で状態を維持することができます。ただしKerasのRNNはデフォルトではステートレス(stateless）であり、各バッチごとにモデル内部に保存してある隠れ状態をリセットしています（厳密にはステートレスとはモデルに状態を保持しないことをいう）。そのため明示的にステートフルに設定する必要があります。今回はその2通りの結果を比較しましょう。まず最初はデフォルトの設定である、ステートレスなモデルから学習を行います。

In [23]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [25]:
from tensorflow.keras.layers import LSTM, Dense

NNの設計でこれまでとは異なる点はLSTM層を用いているところです。
また`RepeatVector`および`TimeDistributed`をインポートしておく必要があります。
`RepeatVector`とは、出力の最大シーケンス長分だけ入力を繰り返す処理を行い、
`TimeDistributed`は時系列に沿って層を結合する処理を担っています。

In [27]:
from tensorflow.keras.layers import RepeatVector, TimeDistributed, Activation

In [28]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128

In [29]:
model = Sequential()

# Encoder
model.add(LSTM(HIDDEN_SIZE, input_shape=(5, 12)))

# Decoder
model.add(RepeatVector(DIGITS + 1))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
#  TensorBoardコールバックを作成する。
from tensorflow.keras.callbacks import TensorBoard
tbcb = TensorBoard(log_dir='./graph_seq2seq_stateless', histogram_freq=1, write_graph=True)

In [32]:
model.fit(train_x, train_t,
                  batch_size=BATCH_SIZE,
                  epochs=100,
                  validation_data=(val_x, val_t),
                  callbacks=[tbcb])
model.reset_states()

Train on 4500 samples, validate on 500 samples
Epoch 1/100
4500/4500 [==============================] - 5s 1ms/sample - loss: 1.9183 - accuracy: 0.2445 - val_loss: 1.7166 - val_accuracy: 0.3933
Epoch 2/100
4500/4500 [==============================] - 0s 64us/sample - loss: 1.6889 - accuracy: 0.2971 - val_loss: 1.6484 - val_accuracy: 0.3660
Epoch 3/100
4500/4500 [==============================] - 0s 67us/sample - loss: 1.6492 - accuracy: 0.3504 - val_loss: 1.6282 - val_accuracy: 0.3813
Epoch 4/100
4500/4500 [==============================] - 0s 65us/sample - loss: 1.6336 - accuracy: 0.3319 - val_loss: 1.6232 - val_accuracy: 0.2573
Epoch 5/100
4500/4500 [==============================] - 0s 65us/sample - loss: 1.6184 - accuracy: 0.3419 - val_loss: 1.5981 - val_accuracy: 0.3600
Epoch 6/100
4500/4500 [==============================] - 0s 65us/sample - loss: 1.5991 - accuracy: 0.3490 - val_loss: 1.5614 - val_accuracy: 0.2653
Epoch 7/100
4500/4500 [==============================] - 0s 65us/s

Epoch 56/100
4500/4500 [==============================] - 0s 66us/sample - loss: 0.6845 - accuracy: 0.6684 - val_loss: 0.7228 - val_accuracy: 0.6293
Epoch 57/100
4500/4500 [==============================] - 0s 64us/sample - loss: 0.6666 - accuracy: 0.6704 - val_loss: 0.6845 - val_accuracy: 0.6660
Epoch 58/100
4500/4500 [==============================] - 0s 66us/sample - loss: 0.6359 - accuracy: 0.6939 - val_loss: 0.6569 - val_accuracy: 0.6793
Epoch 59/100
4500/4500 [==============================] - 0s 67us/sample - loss: 0.6176 - accuracy: 0.6942 - val_loss: 0.6412 - val_accuracy: 0.6833
Epoch 60/100
4500/4500 [==============================] - 0s 67us/sample - loss: 0.5924 - accuracy: 0.7132 - val_loss: 0.6324 - val_accuracy: 0.6740
Epoch 61/100
4500/4500 [==============================] - 0s 66us/sample - loss: 0.5752 - accuracy: 0.7155 - val_loss: 0.6070 - val_accuracy: 0.6933
Epoch 62/100
4500/4500 [==============================] - 0s 65us/sample - loss: 0.5538 - accuracy: 0.7199

### モデルの定義（stateful）
ステートフルモデルを用いる際に注意すべき点が3点あります。
* データの周期性を反映したバッチサイズ
* エポック数だけfor文のループを回す
* データを順番に入力する

In [34]:
np.random.seed(0)

model = Sequential()
model.add(LSTM(HIDDEN_SIZE, stateful=True,
                   batch_input_shape=(BATCH_SIZE, MAXLEN, len(chars)),
                   return_sequences=False))
model.add(layers.RepeatVector(DIGITS + 1))

for _ in range(5):
    model.add(LSTM(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [35]:
train_size = (train_x.shape[0] // BATCH_SIZE) * BATCH_SIZE
test_size = (val_x.shape[0] // BATCH_SIZE) * BATCH_SIZE
train_x, train_t = train_x[:train_size], train_t[:train_size]
val_x, val_t = val_x[:test_size], val_t[:test_size]

In [36]:
tbcb = TensorBoard(log_dir='./graph_seq2seq_stateful', histogram_freq=1, write_graph=True)

NUM_EPOCHS = 100

for i in range(NUM_EPOCHS):
    print("Epoch {:d} / {:d}".format(i + 1, NUM_EPOCHS))
    model.fit(train_x, train_t,
                      batch_size=BATCH_SIZE,
                      epochs=1,
                      validation_data=(val_x, val_t))
    model.reset_states()

Epoch 1 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 11s 3ms/sample - loss: 1.9860 - accuracy: 0.2482 - val_loss: 1.8280 - val_accuracy: 0.2179
Epoch 2 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 158us/sample - loss: 1.7362 - accuracy: 0.2420 - val_loss: 1.6912 - val_accuracy: 0.2344
Epoch 3 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 158us/sample - loss: 1.6983 - accuracy: 0.2620 - val_loss: 1.6907 - val_accuracy: 0.2153
Epoch 4 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 160us/sample - loss: 1.6945 - accuracy: 0.2269 - val_loss: 1.6921 - val_accuracy: 0.2153
Epoch 5 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 158us/sample - loss: 1.6943 - accuracy: 0.2385 - val_loss: 1.6884 - val_accuracy: 0.4002
Epoch 6 / 1

4480/4480 [==============================] - 1s 157us/sample - loss: 1.6899 - accuracy: 0.2739 - val_loss: 1.6832 - val_accuracy: 0.2248
Epoch 43 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 158us/sample - loss: 1.6894 - accuracy: 0.2360 - val_loss: 1.6837 - val_accuracy: 0.2248
Epoch 44 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 159us/sample - loss: 1.6897 - accuracy: 0.3051 - val_loss: 1.6797 - val_accuracy: 0.2248
Epoch 45 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 158us/sample - loss: 1.6902 - accuracy: 0.2539 - val_loss: 1.6824 - val_accuracy: 0.4002
Epoch 46 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 158us/sample - loss: 1.6893 - accuracy: 0.2911 - val_loss: 1.6817 - val_accuracy: 0.4002
Epoch 47 / 100
Train on 4480 samples, validate on 384 samples
4480/

4480/4480 [==============================] - 1s 157us/sample - loss: 0.7733 - accuracy: 0.5981 - val_loss: 0.7979 - val_accuracy: 0.5842
Epoch 84 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 158us/sample - loss: 0.7716 - accuracy: 0.6032 - val_loss: 0.7252 - val_accuracy: 0.6024
Epoch 85 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 159us/sample - loss: 0.7895 - accuracy: 0.5897 - val_loss: 0.7603 - val_accuracy: 0.6085
Epoch 86 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 156us/sample - loss: 0.7878 - accuracy: 0.5943 - val_loss: 0.7865 - val_accuracy: 0.5981
Epoch 87 / 100
Train on 4480 samples, validate on 384 samples
4480/4480 [==============================] - 1s 155us/sample - loss: 0.7659 - accuracy: 0.6027 - val_loss: 0.6794 - val_accuracy: 0.6432
Epoch 88 / 100
Train on 4480 samples, validate on 384 samples
4480/

## 結果の確認
10題ほどランダムに問題を出してその結果を観察しましょう。
まずランダムに検証用のデータからランダムにidを選びます。

In [37]:
ind = np.random.randint(0, len(val_x))
ind

172

続いてidに対応する`x`と`t`を`rowx`と`rowt`とします。

In [43]:
rowx, rowt = val_x, val_t
rowx

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

  

In [44]:
rowt

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

入力`rowx`に対して`model.predict_classes()`とすると予測結果がidで返ってきます。

In [45]:
preds = model.predict_classes(rowx, batch_size=128 ,verbose=0)
preds

array([[1, 2, 2],
       [9, 0, 0],
       [9, 7, 0],
       ...,
       [9, 9, 0],
       [8, 3, 1],
       [9, 6, 0]])

デコード関数を作成します。

In [46]:
def decode(x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(id2word[x] for x in x)

In [56]:
q = decode(rowx[0])
q

'45+79'

In [48]:
correct = decode(rowt[0])
correct

'124'

In [49]:
guess = decode(preds[0], calc_argmax=False)
guess

'122'

In [58]:
q = decode(rowx[1])
q

'64+28'

In [59]:
correct = decode(rowt[1])
correct

'920'

In [60]:
guess = decode(preds[1], calc_argmax=False)
guess

'910'